# Making A Realistic Fuel Assembly 

Previously made an "all fuel" W17$\times$17 fuel assembly

  ![all pins fuel assembly plot](figs/all_pins.png)


# Not realistic
* Should have cells without fuel for
   * Control rod guide tubes
   * Instrumentation (e.g., fission chamber) guide tubes

# Goals
1. Generate guide tube and instrument cells
2. Add said cells to own universe
3. Setup matrix based fill
    * Fill matrix with identical universe
    * Add special universes for Guide tubes and instrument tubes
4. Set importances to 1.0!


# Assembly Layout

![Pin layout of a westinghouse 17x17](figs/w17x17_layout.png)
* Image from Horelik et al. (BEAVRS)<sup>1</sup>; use allowed under MIT license

<sup>1</sup> N. E. Horelik et al., "Benchmark for Evaluation and Validation of Reactor Simulations (BEAVRS)," presented at the Int. Conf. Mathematics and Computational Methods Applied to Nuc. Sci. & Eng., Sun Valley, Idaho, 2013.

# Start
* Import modules
* Read in previous model (use `models/oops_all_pins_ans.imcnp` if needed)

In [ ]:
from _config import install_montepy
install_montepy()

import montepy
import numpy as np
from IPython.display import IFrame
import warnings

problem = montepy.read_input("models/oops_all_pins_ans.imcnp")
warnings.simplefilter("ignore", montepy.errors.LineExpansionWarning)
warnings.simplefilter("ignore", DeprecationWarning)

# Step 1.1: Create Guide Tube Unit cell
1. Create Surfaces for the zircaloy boundaries (use cloning)
2. Create the zircaloy cell
3. Create the moderator cell
4. Add to universe

![Guide pin plan view](figs/guide_pin.png)

* Image from BEAVRS

# Step 1.1.1 Make the surfaces

1. Clone the cylinder
2. Update the radius

In [ ]:
GUIDE_IR = 0.56134  # [cm]
GUIDE_OR = 0.60198  # [cm]
ZIRCALOY_DENSITY = 6.55  # [g/cm3]
WATER_DENSITY = 0.74  # [g/cm3]
base_cyl = list(problem.surfaces.cz)[0]  # grabbing a cylinder to clone

guide_ir_cyl = base_cyl.clone()
guide_ir_cyl.radius = GUIDE_IR

guide_or_cyl = base_cyl.clone()
guide_or_cyl.radius = GUIDE_OR

# Step 1.1.2 Make the Zircaloy cell

1. Make the cell
    1. request number
    2. append to cells
2. Set the geometry (will be infinite in `z`)
3. Set material and density
    1. Find zircaloy material by its `Zr`

# Step 1.1.3 Make the Moderator Cell
1. Make the cell
2. Define the geometry
    * We will just complement the guide tube for simplicity (`~`)
3. Set the material and density

In [ ]:
guide_tube = montepy.Cell()
guide_tube.number = problem.cells.request_number()
problem.cells.append(guide_tube)
# set geometry
guide_tube.geometry = +guide_ir_cyl & -guide_or_cyl

# find zircaloy
zircaloy = list(problem.materials.get_containing_all("Zr"))[0]
# set material and density
guide_tube.material = zircaloy
guide_tube.mass_density = ZIRCALOY_DENSITY


guide_mod = montepy.Cell()
guide_mod.number = problem.cells.request_number()
problem.cells.append(guide_mod)

guide_mod.geometry = ~guide_tube

# grab water
water = list(problem.materials.get_containing_all("H", "O"))[0]
guide_mod.material = water
guide_mod.mass_density = WATER_DENSITY

# Step 1.1.4 Add All Cells to a Universe

* Need to have a unique Universe number
    * Use [`Universes.request_number`](https://www.montepy.org/en/stable/api/montepy.universes.html#montepy.universes.Universes.request_number) for this
* Then [`claim`](https://www.montepy.org/en/stable/api/montepy.universe.html#montepy.universe.Universe.claim) the cells
* Add the universe to the problem

In [ ]:
guide_universe = montepy.Universe(problem.universes.request_number())
guide_universe.claim([guide_tube, guide_mod])
problem.universes.append(guide_universe)

# Step 1.2: Create Instrument Tube Unit cell
1. Create Surfaces for the zircaloy boundaries (use cloning)
2. Create air material
3. create cells
    1. Create the two zircaloy cell
    5. Create the air cell
    6. Create the moderator cell
7. Add to universe

![instrument pin plan view](figs/instrument_tube.png)
* Figure taken from BEAVRS

# Step 1.2.1 Make the Surfaces
* Will use an iterative design this time

In [ ]:
INST_RADII = {
    "PV_IR": 0.4369,
    "PV_OR": 0.4839,
    "TUBE_IR": 0.56134,
    "TUBE_OR": 0.6020,
}  # [cm]

inst_cyls = {}
for cyl_name, cyl_radius in INST_RADII.items():
    cyl = base_cyl.clone()
    cyl.radius = cyl_radius
    inst_cyls[cyl_name] = cyl

# Step 1.2.2 Make Air material
* data taken from BEAVRS
* Given nuclide, atomic density mapping from reference

In [ ]:
AIR_DENSITY = 0.00616  # [g/cm3]
air_comp = {
    "Ar-36": 7.8730e-09,
    "Ar-38": 1.4844e-09,
    "Ar-40": 2.3506e-06,
    "C-12": 6.7539e-08,
    "C-13": 7.5658e-10,
    "N-14": 1.9680e-04,
    "N-15": 7.2354e-07,
    "O-16": 5.2866e-05,
    "O-17": 2.0084e-08,
    "O-18": 1.0601e-07,
}  # data are in a/b-cm

NOT_IN_ENDF_VII = {"O-18"}

## Need to clean up data for use with ENDF/B-VII.1
* Has isotopic Carbon
    * need to combine for `C-nat` in ENDF/B-VII.1
* Has nuclides (O-18) not in ENDF/B-VII.1 
    * need to remove them
 
## Steps
1. Remove `NOT_IN_ENDF_VII` from `air_comp`
2. iterate over the dictionary with [`items`](https://docs.python.org/3/tutorial/datastructures.html)
   1. Note which nuclides are carbon
   2. add up the total atomic density of carbon
3. Delete all carbon nuclides from `air_comp`
4. Add elemental carbon to `air_comp`

In [ ]:
# remove extra nuclides
for bad_nuclide in NOT_IN_ENDF_VII:
    del air_comp[bad_nuclide]

# find Carbon nuclides:
carbon_nuclides = set()
carbon_frac = 0.0
for nuclide, fraction in air_comp.items():
    if "C-" in nuclide:
        carbon_nuclides.add(nuclide)
        carbon_frac += fraction

for nuclide in carbon_nuclides:
    del air_comp[nuclide]

air_comp["C-0"] = carbon_frac
air_comp

# Make the Material now

* Use library `82c` with all nuclides

## Steps
1. Make the material (and append it)
2. Add all components from `air_comp`
    * see [`add_nuclide`](https://www.montepy.org/en/stable/api/montepy.data_inputs.material.html#montepy.data_inputs.material.Material.add_nuclide)
3. [Renormalize](https://www.montepy.org/en/stable/api/montepy.data_inputs.material.html#montepy.data_inputs.material.Material.normalize) the composition
    * This is a matter of personal preference

In [ ]:
air_mat = montepy.Material(number=problem.materials.request_number())
problem.materials.append(air_mat)

# add the nuclides from the dictionary
for nuclide, fraction in air_comp.items():
    air_mat.add_nuclide(f"{nuclide}.82c", fraction)

# renormalize
air_mat.normalize()
print(air_mat)
air_mat

# Step 1.2.3 Make the cells

* Will use an iterative approach again
* A for loop will be given to help you
* Steps
    1. Make cell
    2. add to problem
    3. assign material
    4. assign density
    5. assign geometry
    6. append to list

In [ ]:
# define the cells by their material and density
inst_materials = [
    # Material   Density
    (air_mat, AIR_DENSITY),
    (zircaloy, ZIRCALOY_DENSITY),
    (water, WATER_DENSITY),
    (zircaloy, ZIRCALOY_DENSITY),
    (water, WATER_DENSITY),
]

## Make the cells
* Steps
    1. Make cell
    2. add to problem
    3. assign material
    4. assign density
    5. assign geometry
    6. append to list

# Task: Create the 5 cells for the instrument tube

In [ ]:
# Task: Use the 'for' loop to
inst_cells = []
for (mat, density), inner_rad, outer_rad in zip(
    inst_materials, [None] + list(inst_cyls.values()), list(inst_cyls.values()) + [None]
):
    print(mat, density, inner_rad, outer_rad)
    # Make a new cell
    cell = montepy.Cell(number=problem.cells.request_number())
    # Then set the material, density, and geometry
    cell.material = None
    cell.mass_density = None
    cell.geometry = None
    print(cell)
    # When ready
    # inst_cells.append(cell)

In [ ]:
problem.cells.extend(inst_cells)

# Step 1.2.4 Move All Cells to Own Universe

## steps
1. Make the universe
2. Claim all relevant cells

In [ ]:
inst_universe = montepy.Universe(problem.universes.request_number())
inst_universe.claim(inst_cells)
problem.universes.append(inst_universe)
inst_universe

# Step 3 Setup multi-universe matrix fill
1. Grab lattice filled cell
2. Make matrix of universes
3. Add guide and instrument pins

# Step 3.1 Grab Lattice fill cell and the fuel universe
* It is the only cell with a non-None `cell.lattice`.

In [ ]:
for cell in problem.cells:
    if cell.lattice is not None:
        lattice_cell = cell
        break
print(lattice_cell)
fuel_universe = problem.universes[100]
print(fuel_universe)

# Step 3.3 Make a universe Matrix
* Need a 17$\times$17$\times$1 array of universes
* Use `numpy.full` to build this

In [ ]:
IFrame("https://numpy.org/doc/stable/reference/generated/numpy.full.html", 800, 600)

In [ ]:
fill_matrix = np.full((17, 17, 1), fuel_universe)
print(fill_matrix.shape)
print("Preview:")
print(fill_matrix[:3, :3, 0])

# Step 4: Add guide and Instrument Tubes
* Just need to change a few elements of the matrix now.
* Assembly has 8 fold rotational symmetry
* a 4-fold symmetry preserving function is provided
   * Implementing 8-fold symmetry is an exercise left to the reader

In [ ]:
def set_w17_17_universe(uni_matrix, universe, x_idx, y_idx):
    """
    Sets the given index in the matrix to the given universe while also preserving 4-fold rotational symmetry.

    Parameters
    ----------
    uni_matrix: numpy.ndarray
        The 3d array of universes representing pin cells in a W17x17
    universe: montepy.Universe
        The universe to fill in at the given index
    x_idx: int
        The index in the x dimension of the cell to fill with the given universe. (0,0) is bottom left corner
    y_idx: int
        the index in the y dimension of the cell to fill with the given universe.
    """
    SYMM_OFFSET = 8
    x_offset = SYMM_OFFSET - x_idx
    y_offset = SYMM_OFFSET - y_idx
    for x_mult, y_mult in [(0, 0), (0, 1), (1, 0), (1, 1)]:
        new_x = (x_idx + 2 * x_mult * x_offset) % 17
        new_y = (y_idx + 2 * y_mult * y_offset) % 17
        uni_matrix[new_x, new_y, 0] = universe

# Step 4.1 Set the Instrument Tube

* Update `fill_matrix` with `inst_universe`

![w17x17 layout positions with instrument tube in 8,8](figs/w17x17_layout_labeled.png)
* base figure from BEAVRS; poorly formatted numbers added by Micah

* Reminder we used `inst_universe`

# Task: Place the instrument tube in the lattice.

In [ ]:
# Task: place an individual instrument tube in the center.
x = None
y = None
if x is not None and y is not None:
    fill_matrix[x, y, 0] = inst_universe

# Step 4.2 Set the Guide Tubes 

* Update `fill_matrix` with `guide_universe`

![w17x17 layout positions with guide tubes at (3,3), (5,2), (5,5), (2,5), (8,2), (8,5), (2,8), (5,8)](figs/w17x17_layout_labeled.png)
* base figure from BEAVRS; poorly formatted numbers added by Micah

## Steps
* Call `set_w17_17_universe` for each guide tube position in the lower left quadrant
* reminder we used `guide_universe`

# Task: Replace some of the Fuel universes with Guide Tube universes

In [ ]:
# Task: Place the guide tubes into the lattice.
# Use the BEAVRS illustration for inspiration, or make your own design!
# Extra credit: write your own set_w17_17_universe() to use octant symmetry, which will require only 5 pairs.
quadrant_pairs = [
    # Pick any quadrant and populate the 8 (x, y) pairs
]
for x, y in quadrant_pairs:
    set_w17_17_universe(fill_matrix, guide_universe, x, y)

# Step 4.3 Set lattice cell fill with this matrix
* Set in [`cell.fill`](https://www.montepy.org/en/stable/api/montepy.cell.html#montepy.cell.Cell.fill)
* Use [`Fill.multiple_universes`](https://www.montepy.org/en/stable/api/montepy.data_inputs.fill.html#montepy.data_inputs.fill.Fill.multiple_universes), and [`Fill.universes`](https://www.montepy.org/en/stable/api/montepy.data_inputs.fill.html#montepy.data_inputs.fill.Fill.universes)

In [ ]:
?lattice_cell.fill
# help(lattice_cell.fill)

In [ ]:
lattice_cell.fill.multiple_universes = True
lattice_cell.fill.universes = fill_matrix

# Step 5: Update importances

* Once again the cell default importance is `0.0`
* Update all importances to be `1.0`
* use [`Cells.set_equal_importance`](https://www.montepy.org/en/stable/api/montepy.cells.html#montepy.cells.Cells.set_equal_importance)

**Note:** The MontePy docs use the term "vacuum" cell to refer to a zero-importance cell, which immediately kills the particle in MCNP. This is not the same as a "void" cell.

In [ ]:
# Task: Set the importance of every cell to 1.0
?problem.cells.set_equal_importance
# Extra credit: set the importance of the instrumentation to zero (total absorption)

# Conclusion

* Write out to file `models/w17_17_k_inf.imcnp`

In [ ]:
problem.write_problem("models/w17_17_k_inf.imcnp", overwrite=True)

# Results: Plots

![overall westinghouse 17x17 fuel assembly plot](figs/w17x17_plot.png)

## Instrument Cell

![a focus on the instrument pin cell](figs/w17x17_instrument.png)

# Guide Tube Cell

## Questions?

![A focused view of the a guide tube](figs/w17x17_guide.png)